In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

pd.options.display.max_rows = 4
pd.options.display.max_colwidth = 100

In [2]:
prods = pd.read_csv('product_descriptions.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
attributes = pd.read_csv('attributes.csv')
#TODO : Havent' used attributes.csv at all

In [3]:
#product_descriptions.csv doesn't have the "product_title" column. Augmenting it with the values from the full dataset.
train_uids = train.filter(items=['product_uid', 'product_title'])
test_uids = test.filter(items=['product_uid', 'product_title'])
data_uids = train_uids.merge(test_uids, how = "outer", on = 'product_uid')

data_uids['product_title'] = data_uids['product_title_x']
nas = data_uids['product_title'].isnull()
#The below statement throws a SettingWithCopyWarning because of the [a][b] indexing. Its safe in this case.
data_uids['product_title'][nas] = data_uids['product_title_y'][nas]
del data_uids['product_title_x'], data_uids['product_title_y']
data_uids.drop_duplicates(subset=['product_uid', 'product_title'], inplace=True)

prods = prods.merge(data_uids, on="product_uid")
prods['full_description'] = prods['product_title'] + ". " + prods['product_description']
print(map(len, [np.unique(df['product_uid'].values) for df in [prods, data_uids, train, test]]))

#We have title dupes
print(len(np.unique(data_uids['product_uid'].values)))
print(len(np.unique(data_uids['product_title'].values)))

[124428, 124428, 54667, 97460]
124428
120348


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
pd.options.display.max_rows = 50
attributes[attributes['product_uid']==100002]

,product_uid,name,value
15,100002,Application Method,"Brush,Roller,Spray"
16,100002,Assembled Depth (in.),6.63 in
17,100002,Assembled Height (in.),7.76 in
18,100002,Assembled Width (in.),6.63 in
19,100002,Bullet01,"Revives wood and composite decks, railings, porches and boat docks, also great for concrete pool..."
20,100002,Bullet02,100% acrylic solid color coating
21,100002,Bullet03,Resists cracking and peeling and conceals splinters and cracks up to 1/4 in.
22,100002,Bullet04,"Provides a durable, mildew resistant finish"
23,100002,Bullet05,Covers up to 75 sq. ft. in 2 coats per gallon
24,100002,Bullet06,"Creates a textured, slip-resistant finish"


In [65]:
#NOTE: The word "brand" in the name usually is about the mfg brand.
#However, there are some entries about fit, compatibility and "pump brand", whatever that is.

brand_name_predicate = attributes['name'].str.lower().str.contains("brand").fillna(False)
brand_names = attributes[brand_name_predicate]
print brand_names.shape

mfg = brand_names['name'].str.lower().str.contains("mfg").fillna(False)
print brand_names[mfg].shape

fit = brand_names['name'].str.lower().str.contains("fit").fillna(False)
print brand_names[fit].shape

compat = brand_names['name'].str.lower().str.contains("compat").fillna(False)
print brand_names[compat].shape

pump = brand_names['name'].str.lower().str.contains("pump").fillna(False)
print brand_names[pump].shape

other = brand_names[~(mfg | fit | compat | pump)]
print other.shape

(88881, 3)
(86250, 3)
(1250, 3)
(1294, 3)
(87, 3)
(0, 3)
